In [174]:
'''
!!!МЕНЯЕМ ТОЛЬКО ЭТУ ЯЧЕЙКУ!!! ДАЛЬШЕ НИЧЕГО НЕ МЕНЯЕМ !!!
PS для более тонкой настройки можно менять параметры модели в блоке обучения модели(ниже)

Блок параметров обучения модели
'''
rnn1_utils = 1200
lstm1_units = 1000
dense_units = 500
epochs = 20
batch_size = 52
activation = 'relu'
optimizer = 'adam'
# rnn1_utils = 1200
# lstm1_units = 1000
# dense_units = 500
# epochs = 2000
# batch_size = 52
# activation = 'relu'
# optimizer = 'adam'


In [175]:
end_future_date = '2024-12-31'

In [176]:
import pandas as pd

df_Pl = pd.read_csv('/Users/nikitasavvin/HSE_work/clustering-ru-terms/initial data/data-Сенсорномоторная координация и пространственное позиционирование.csv', sep=';')


In [177]:
df_Pl

,pubdate,count
0,1999-12-27,16
1,2000-01-03,6
2,2000-01-10,12
3,2000-01-17,17
4,2000-01-24,2
...,...,...
1243,2023-11-13,22
1244,2023-11-20,13
1245,2023-11-27,17
1246,2023-12-04,12


In [178]:
# Узнаем последнию известную дату. Нужна для создания промежутка значений будущих дат
end_know_date = df_Pl['pubdate'].iloc[-1]

end_know_date_index = df_Pl[df_Pl['pubdate'] == end_know_date].index[0] + 1


In [179]:
import pandas as pd
import numpy as np

# Создаем будущие даты с шагом в 7 дней
future_dates = pd.date_range(start=end_know_date, end=end_future_date, freq='7D')

future_df = pd.DataFrame({'pubdate': future_dates, 'count': np.nan})

avg_pl = df_Pl['count'].mean()

df_Pl['count'].fillna(avg_pl, inplace=True)

df_Pl_sacler = pd.concat([df_Pl, future_df], ignore_index=True)


In [180]:
df_Pl_sacler

,pubdate,count
0,1999-12-27,16.0
1,2000-01-03,6.0
2,2000-01-10,12.0
3,2000-01-17,17.0
4,2000-01-24,2.0
...,...,...
1299,2024-12-02 00:00:00,NaN
1300,2024-12-09 00:00:00,NaN
1301,2024-12-16 00:00:00,NaN
1302,2024-12-23 00:00:00,NaN


In [181]:
from utils.normalization import TimeNormalization

# блок нормировки дат относительно всех дат(известных и которых хотим прогнощировать)
lower_limit = pd.Timestamp(df_Pl_sacler['pubdate'].iloc[0])
upper_limit = pd.Timestamp(df_Pl_sacler['pubdate'].iloc[-1])

lower_limit = lower_limit.strftime('%Y-%m-%d')
lower_limit = str(lower_limit)
upper_limit = upper_limit.strftime('%Y-%m-%d')
upper_limit = str(upper_limit)

TimeScaller = TimeNormalization(
                lower_limit=lower_limit,
                upper_limit=upper_limit,
                time_format="%Y-%m-%d"
            )


In [182]:
df_Pl_sacler

,pubdate,count
0,1999-12-27,16.0
1,2000-01-03,6.0
2,2000-01-10,12.0
3,2000-01-17,17.0
4,2000-01-24,2.0
...,...,...
1299,2024-12-02 00:00:00,NaN
1300,2024-12-09 00:00:00,NaN
1301,2024-12-16 00:00:00,NaN
1302,2024-12-23 00:00:00,NaN


In [183]:
df_Pl_sacler['pubdate'] = pd.to_datetime(df_Pl_sacler['pubdate'])

dates = df_Pl_sacler['pubdate'].dt.date.astype(str).to_list()

sep_dates = TimeScaller.separation(dates)

list_norm_dates = TimeScaller.normalize(sep_dates)


1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999
1999


In [184]:
df_Pl_normalized_all_dates = pd.DataFrame(list_norm_dates, columns=['year', 'month', 'day'])

In [185]:
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()

count_column = df_Pl_sacler[['count']]

count_normalized = scaler.fit_transform(count_column)

df_Pl_normalized_all_dates['count'] = count_normalized

In [186]:
df_Pl_normalized_all_dates

,year,month,day,count
0,0.00,1.0,0.866667,0.052265
1,0.04,0.0,0.066667,0.017422
2,0.04,0.0,0.300000,0.038328
3,0.04,0.0,0.533333,0.055749
4,0.04,0.0,0.766667,0.003484
...,...,...,...,...
1299,1.00,1.0,0.033333,NaN
1300,1.00,1.0,0.266667,NaN
1301,1.00,1.0,0.500000,NaN
1302,1.00,1.0,0.733333,NaN


In [187]:

df_Pl_normalized_initial_dates = df_Pl_normalized_all_dates.loc[: end_know_date_index-1]


In [188]:
df_Pl_normalized_future_dates = df_Pl_normalized_all_dates.loc[end_know_date_index :]


In [189]:
df_Pl_normalized_future_dates

,year,month,day,count
1248,0.96,1.000000,0.333333,NaN
1249,0.96,1.000000,0.566667,NaN
1250,0.96,1.000000,0.800000,NaN
1251,1.00,0.000000,0.000000,NaN
1252,1.00,0.000000,0.233333,NaN
1253,1.00,0.000000,0.466667,NaN
1254,1.00,0.000000,0.700000,NaN
1255,1.00,0.000000,0.933333,NaN
1256,1.00,0.090909,0.142857,NaN
1257,1.00,0.090909,0.392857,NaN


In [190]:
df_Pl_normalized_future_dates

,year,month,day,count
1248,0.96,1.000000,0.333333,NaN
1249,0.96,1.000000,0.566667,NaN
1250,0.96,1.000000,0.800000,NaN
1251,1.00,0.000000,0.000000,NaN
1252,1.00,0.000000,0.233333,NaN
1253,1.00,0.000000,0.466667,NaN
1254,1.00,0.000000,0.700000,NaN
1255,1.00,0.000000,0.933333,NaN
1256,1.00,0.090909,0.142857,NaN
1257,1.00,0.090909,0.392857,NaN


In [191]:
# Данные для обучения
X_initial = df_Pl_normalized_initial_dates[['year', 'month', 'day']].values
y_initial = df_Pl_normalized_initial_dates['count'].values


In [192]:

X_initial_train = X_initial.reshape(-1, 1, 3)


In [193]:
from sklearn.model_selection import train_test_split

X_initial = df_Pl_normalized_initial_dates[['year', 'month', 'day']].values
y_initial = df_Pl_normalized_initial_dates['count'].values

X_initial_train = X_initial.reshape(-1, 1, 3)

X_train, X_test, y_train, y_test = train_test_split(X_initial_train, y_initial, test_size=0.01, random_state=42)


In [194]:
'''
Блок обучения модели
'''
#можно менять параметры активации количество слоев и прочее

from tensorflow.keras.layers import Input, LSTM, Dense, Conv1D, MaxPooling1D, Flatten, concatenate, SimpleRNN, Dropout
from tensorflow.keras.models import Model

# Создание входов
input_lstm = Input(shape=(1, 3), name='input_lstm')
input_rnn = Input(shape=(1, 3), name='input_rnn')


rnn1 = SimpleRNN(units=rnn1_utils, activation=activation)(input_rnn)
rnn1 = Dropout(0.05)(rnn1)
lstm1 = LSTM(units=lstm1_units, activation=activation)(input_lstm)
lstm1 = Dropout(0.05)(lstm1)

# Объединение выходов сверточного слоя и выходов для времени
merged = concatenate([rnn1, lstm1])

dense = Dense(units=dense_units, activation=activation)(merged)

output = Dense(units=1, activation=activation)(dense)

model = Model(inputs=[input_rnn, input_lstm], outputs=output)
model.compile(optimizer=optimizer, loss='mean_squared_error')

epochs = epochs
batch_size = batch_size
model.fit([X_train, X_train], y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/20
24/24 [==============================] - 1s 15ms/step - loss: 0.0138
Epoch 2/20
24/24 [==============================] - 0s 16ms/step - loss: 0.0085
Epoch 3/20
24/24 [==============================] - 0s 17ms/step - loss: 0.0080
Epoch 4/20
24/24 [==============================] - 0s 15ms/step - loss: 0.0072
Epoch 5/20
24/24 [==============================] - 0s 16ms/step - loss: 0.0065
Epoch 6/20
24/24 [==============================] - 0s 16ms/step - loss: 0.0062
Epoch 7/20
24/24 [==============================] - 0s 15ms/step - loss: 0.0054
Epoch 8/20
24/24 [==============================] - 0s 15ms/step - loss: 0.0050
Epoch 9/20
24/24 [==============================] - 0s 16ms/step - loss: 0.0046
Epoch 10/20
24/24 [==============================] - 0s 18ms/step - loss: 0.0043
Epoch 11/20
24/24 [==============================] - 0s 14ms/step - loss: 0.0042
Epoch 12/20
24/24 [==============================] - 0s 15ms/step - loss: 0.0041
Epoch 13/20
24/24 [==================

In [195]:
'''
Нейминг моделей
model_c{count}_nc{cont}_l{count}_nl{count}_e{count}_bs{count}
где c{count} - количество cnn слоев
nc{cont} -  количество нейронов cnn слоев (если несколько то через _)
l{count} - количество lstm слоев
nl{count} - количество нейронов lstm слоев (если несколько то через _)
e{count} - количество эпох обучения
bs{count} - batch_size
'''
import os


home_path_models = os.path.abspath('models')


model_name = f'model_predictor_work_1.h5'

model.save(f'/Users/nikitasavvin/HSE_work/clustering-ru-terms/models/{model_name}')


/Users/nikitasavvin/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [196]:
model_name

'model_predictor_work_1.h5'